In [1]:
import pandas as pd
import numpy as np
import json
import codecs
import re
import dateparser

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def extract_int_data(input_str):
    data = {}  # Use a dictionary to store extracted data
    nxt = {
        '"nombreLikesArticle"': '"nombreLecturesArticle"',
        '"nombreLecturesArticle"': '"nombreCommentairesArticle"',
        '"nombreCommentairesArticle"': '"nombrePartagesArticle"',
        '"nombrePartagesArticle"': '"contenuArticle"'
    }

    for key in nxt.keys():
        pattern = f'{key}:\\s*(.*?),' + f'{nxt[key]}'
        match = re.search(pattern, input_str)

        if match:
            data[key] = match.group(1).replace(" ", "")
            try:
                data[key] = int(data[key])
            except ValueError:
                data[key] = 0

    # Replace data in the input string
    for key, value in data.items():
        input_str = re.sub(f'{key}:.*?,{nxt[key]}', f'{key}:{value},{nxt[key]}', input_str)

    return input_str

In [7]:
def process_video_data(strng):
    
    pattern = r'"videosArticle":\s*(.*?)"nombreLikesArticle"'
    match = re.search(pattern, strng)
    
    if not match: return strng
    data = match.group(1)
    # Retirez la virgule finale si elle existe
    if data.endswith(','):
        data = data[:-1]
    # print("Initial data:", data)
    
    if data == "[]": return strng
    
    # Retirez les espaces de début et de fin
    data = data.strip()
    
    while data.startswith("[,"):
        # Correction de la chaîne "[,"
        data = "[" + data[2:]
    
    while data.endswith(",]"):
        # Correction de la chaîne "[,"
        data = data[:-2] + "]"
    
    if data.count('"') < 2 and len(data) != 2:
        # Traitement pour s'assurer qu'il y a au moins deux guillemets
        data = '["' + data[1:-1] + '"]'
    
    if not data.startswith("["):
        data = '[' + data
    
    if not data.endswith("]"):
        data += ']'
    
    # Vérifiez la validité des données
    # if data.count('"') % 2 != 0 and data.count(',') > 0: pass
        # print("Traitement nécessaire pour les données:", data)  
    
    if data.count('"') % 2 == 0 and '""https://' in data:
        # print("Got It")
        data = data.replace('""https://','","https://')
        

    strng = re.sub(pattern, f'"videosArticle":{data},"nombreLikesArticle"', strng)
    return strng

In [10]:
def extract_artcle_id(strng) :
    match = re.search(r'"_id":\s*\d+', strng)
    captured_number = match.group(0)
    return captured_number

In [13]:
i = 0
processed_lines = []
with codecs.open('data/baseArticles.json', 'r', encoding='utf-8') as f:
    for line in f:
        if line.startswith('{"index":{"'): 
            article_id = extract_artcle_id(line)
            continue
        try:
            line = line.replace('\u00A0', ' ') # Remove non-breaking space characters (U+00A0)
            line = extract_int_data(line)  # treat_numerical_data(line)
            cleaned_line = process_video_data(line)
            cleaned_line = '{'+ f'{article_id},{cleaned_line[1:]}'
            cleaned = json.loads(cleaned_line)
            processed_lines.append(cleaned)
        except json.JSONDecodeError as e:
            i += 1 

print(f'Errors number : {i}')

Errors number : 19


In [16]:
nosql_db = json.dumps(processed_lines, ensure_ascii=False) # , cls=NpEncoder
jsonFile = open("data/processed_baseArticles.json", "w", encoding='utf-8')
jsonFile.write(nosql_db)
jsonFile.close()

In [18]:
df = pd.read_json("data/processed_baseArticles.json")

In [19]:
df.head(1)

,_id,identifiantArticle,urlArticle,sourceArticle,thematiqueArticle,sousCategorieArticle,imagesArticle,titreArticle,resumeArticle,auteurArticle,...,dateMiseJourArticle,videosArticle,nombreLikesArticle,nombreLecturesArticle,nombreCommentairesArticle,nombrePartagesArticle,contenuArticle,commentairesArticle,dateCollecteArticle,motsClesArticle
0,0,https___www.yerimpost.com_video-voici-samba-di...,https://www.yerimpost.com/video-voici-samba-di...,yerimpost,Sociéte,,[https://www.yerimpost.com/wp-content/uploads/...,"Vidéo- Voici Samba Diagne, le Sénégalais qui a...",Share on: WhatsApp,Khalifa Ndiaye,...,,[https://www.youtube.com/embed/ZKXUmwh5SLc],0,0,0,0,,[],Wed Oct 09 11:28:49 GMT 2019,


In [22]:
def auteur_article_processing(auteur):
    
    # Valider que auteur est une chaîne de caractères  
    if not isinstance(auteur, str):
        return auteur

    pattern = r'\b\d{2}/\d{2}/\d{4}\b'  # Format 'jj/mm/aaaa'
    # Recherche de la date dans la chaîne
    match = re.search(pattern, auteur)
    # Si une date a été trouvée, la remplacer par une chaîne vide
    if match:
        auteur = re.sub(pattern, '', auteur)    

    if auteur.startswith("Par: "):
        auteur = auteur[4:]
           
    if " & " in auteur:
        auteur = auteur.replace(" & ", ", ")
        
    if "Un reportage de " in auteur :
        auteur = auteur.replace("Un reportage de ", "")  
    
    # supprime les addresses email, les titres, la source de l'auteur
    
    if "-" in auteur: 
        auteur = auteur.split("-", 1)[0]
            
    if ";" in auteur:
        auteur = auteur.replace(";", ",")
    
    if ":" in auteur:
        auteur = auteur.split(':')[0]
    
    if " et " in auteur:
        auteur = auteur.replace(" et ", ", ")
        
    if "Propos recueillis par" in auteur : 
        auteur = auteur.replace("Propos recueillis par", "")
    
    if "Entretien réalisé par " in auteur :
        auteur = auteur.replace("Entretien réalisé par ", "")
    
    if "envoyé spécial à" in auteur and "," in auteur : 
        auteur = auteur.split(',', 1)[0] if auteur.find(',') < auteur.find('à') else  auteur.split(',', 1)[1]
    
    if "De notre correspondant" in auteur and "," in auteur :
        auteur = auteur.split(',', 1)[1]
        
    if " de " in auteur:
        auteur = auteur.split(" de ", 1)[0]
        
    if "@" in auteur and "-" in auteur :
        auteur = auteur.split('-')[0]
    
    if "www.facebook.com/" in auteur :
        auteur = ""
    
    # suppression de tout entre parenthèses et les parenthèses elles-mêmes?   
    auteur = re.sub(r'\([^)]*\)', '', auteur)
    
    if "(" in auteur : 
        auteur = auteur.split('(')[0]
    
    # suppression des espaces avant et après chaque virgule
    auteur = re.sub(r'\s*,\s*', ',', auteur)
    
    if len(auteur) <= 1 : return ''
    # suppression des addresses emails
    pattern = r'\S+@\S+'  # Cette expression régulière correspond à une adresse e-mail
    # adresses_email = re.findall(pattern, chaine)
    auteur = re.sub(pattern, '', auteur)
    auteur = auteur.strip().title()
    
    return auteur

In [23]:
def mot_cles_article_processing(keywords):
    keywords = keywords.lower()
    keywords = keywords.replace(";", ",")
    
    while keywords.startswith(","):
        keywords = keywords[1:]
    
    while keywords.endswith(","):
        # Correction de la chaîne "[,"
        keywords = keywords[:-1]    
    keywords = re.sub(r'\s*,\s*', ',', keywords)
    return keywords  

In [24]:
def thematique_processing(thematique):
    thematique = thematique.lower()
    thematique = thematique.replace(";", ",")
    
    while thematique.startswith(","):
        thematique = thematique[1:]
    
    while thematique.endswith(","):
        thematique = thematique[:-1]
    thematique = re.sub(r'\s*,\s*', ',', thematique)
    return thematique

In [25]:
def normalize_date(date_str):
    pattern = r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}'
    match = re.search(pattern, date_str)
    
    if match: # la date est déja au format ISO
        date_obj = dateparser.parse(date_str, languages=['en'])
        iso_format = date_obj.strftime("%Y-%m-%dT%H:%M:%S")
    
    else:
        
        if 'igé par DakarFlash.com, ' in date_str:
            date_str = date_str.replace('igé par DakarFlash.com, ', '')
        
        date_obj = dateparser.parse(date_str, languages=['fr']) # si la chaine est en fr
        if date_obj:
            iso_format = date_obj.strftime("%Y-%m-%dT%H:%M:%S")
        
        else: # dans le cas contraire - en
            date_obj = dateparser.parse(date_str, languages=['en'])
            
            if date_obj :
                iso_format = date_obj.strftime("%Y-%m-%dT%H:%M:%S")
            else : return ''
    
    return iso_format

In [26]:
df["auteurArticle"] = df["auteurArticle"].apply(lambda x: auteur_article_processing(x))

In [27]:
df["datePublicationArticle"] = df["datePublicationArticle"].apply(lambda x: normalize_date(x))

In [28]:
df["dateMiseJourArticle"] = df["dateMiseJourArticle"].apply(lambda x: normalize_date(x))

In [29]:
df["dateCollecteArticle"] = df["dateCollecteArticle"].apply(lambda x: normalize_date(x))

In [30]:
df["motsClesArticle"] = df["motsClesArticle"].apply(lambda x: mot_cles_article_processing(x))

In [31]:
df["thematiqueArticle"] = df["thematiqueArticle"].apply(lambda x: thematique_processing(x)) 

In [ ]:
condition1 = df["commentairesArticle"].apply(len) == 0
condition2 = df["nombreCommentairesArticle"] != 0

In [33]:
df.loc[condition1 & condition2, "nombreCommentairesArticle"] = 0

In [35]:
cnd1 = df["videosArticle"].apply(len) != 0
cnd2 = df["contenuArticle"] == " "
cnd3 = df["contenuArticle"] == ""

In [37]:
# Les articles ayant aucun contenu mais possédant un lien (679, 3)
df.loc[(cnd1 & cnd2) | (cnd1 & cnd3), 'sousCategorieArticle'] = "video"

In [38]:
# Les articles contenant un contenu
df.loc[(~cnd2 & ~cnd3), "sousCategorieArticle"] = "texte"

In [39]:
df.loc[(~cnd2 & ~cnd3), "sousCategorieArticle"].shape

(7041,)

In [40]:
# Ces articles sans contenu, ni lien de vidéos n'ont pas de sousCategorieArticle (1343, 3)

df[(~cnd1 & cnd2) | (~cnd1 & cnd3)][["_id", "videosArticle", "contenuArticle"]].shape

(1343, 3)

In [41]:
def transformation1(commentaire):
    # Utilisez des expressions régulières pour extraire les informations
    numero_match = re.search(r'Commentaire n°(\d+)', commentaire)
    auteur_match = re.search(r'- (.+?) dit :', commentaire)
    date_match = re.search(r'(\d{1,2} \w+ \d{4} à \d{1,2} h \d{1,2} min)', commentaire)
    commentaire_match = re.search(r'min (.+)', commentaire)

    if numero_match and auteur_match and date_match and commentaire_match:
        numero = int(numero_match.group(1))
        auteur = auteur_match.group(1)
        date = date_match.group(1)
        commentaire_texte = commentaire_match.group(1)
        
        if commentaire_texte.endswith("Répondre"): 
            commentaire_texte = commentaire_texte[:-9]
        # Formatez la date au format "09/10/2019 à 12 h 37 min"
        
        date = normalize_date(date)
        
        # Créez un dictionnaire avec les informations extraites
        formatted_comment = {
            'numero': int(numero),
            'auteur': auteur.title(),
            'date': date,
            'commentaire': commentaire_texte
        }
        
        return formatted_comment
    else:
        
        return False

In [42]:
# '### Commentaire n°0 - 1.Posté par LOVEMASSAGE le 07/10/2019 14:09 FINA MASSAGE 781352362 - 761413656 - VOUS DÉSIREZ ME CONNAITRE SUREMENT. MON MASSAGE EST UN MÉLANGE DE SENSUALITÉ ET DE VOLUPTÉ. INTENSÉMENT RELAXANT. A DÉGUSTER SANS MODÉRATION… SATISFACTION ASSURÉE.'
def transformation2(commentaire):
    # Utilisez des expressions régulières pour extraire les informations
    numero_match = re.search(r'Commentaire n°(\d+)', commentaire)
    auteur_match = re.search(r'Posté par (\S+)', commentaire)
    date_match = re.search(r'le (\d{1,2}/\d{1,2}/\d{4} \d{1,2}:\d{2})', commentaire)
    contenu_match = re.search(r'le \d{1,2}/\d{1,2}/\d{4} \d{1,2}:\d{2} (.+)', commentaire)

    # Initialisez un dictionnaire vide
    resultat = {}

    # Si les correspondances sont trouvées, ajoutez les informations au dictionnaire
    if numero_match and auteur_match and date_match and contenu_match:
        resultat['numero'] = int(numero_match.group(1))
        resultat['auteur'] = auteur_match.group(1).title()
        resultat['date'] = normalize_date(date_match.group(1))
        resultat['commentaire'] = contenu_match.group(1)
    
    else:
        return False           

    return resultat

In [43]:
def transformation3(commentaire):
    # Utilisez des expressions régulières pour extraire les informations
    numero_match = re.search(r'Commentaire n°(\d+)', commentaire)
    auteur_date_match = re.search(r'-(.+?) (\d{1,2} \w+ \d{4} at \d{1,2} h \d{2} min)', commentaire)
    contenu_match = re.search(r'at \d{1,2} h \d{2} min (.+)', commentaire)

    # Initialisez un dictionnaire vide
    resultat = {}

    # Si les correspondances sont trouvées, ajoutez les informations au dictionnaire
    if numero_match and auteur_date_match and contenu_match:
        
        resultat['numero'] = int(numero_match.group(1))
        auteur_date = auteur_date_match.group(1).strip()
        resultat['auteur'] = auteur_date.split(" ")[0].title()
        resultat['date'] = normalize_date(auteur_date_match.group(2))
        resultat['commentaire'] = contenu_match.group(1)

    else:
        False
    return resultat

In [44]:
def transformer_commentaire(commentaire):
    
    rslt = transformation1(commentaire) 
    if rslt is not False:
        return rslt
 
    rslt = transformation2(commentaire) 
    if rslt is not False:
        return rslt
    
    rslt = transformation3(commentaire)
    if rslt is not False:
        return rslt
    else :
        return commentaire

In [45]:
def sort_comment_list(liste):
    if len (liste) <= 1: return liste
    if isinstance(liste[0], dict) is False : return liste
    try:
        sorted_liste = sorted(liste, key=lambda x: x['numero'])
    except:
        return liste
    return sorted_liste

In [46]:
# Exemple d'utilisation
commentaire_text = '### Commentaire n°0 - Deugg rék dit : 9 octobre 2019 à 12 h 37 min Mamour walla sonko Amna kou thi am probléme Kane ? Xam ! Na saff na saffa saff rék Et nous on va compter Les coups Répondre'
resultat = transformer_commentaire(commentaire_text)
print(resultat)

{'numero': 0, 'auteur': 'Deugg Rék', 'date': '2019-10-09T12:37:00', 'commentaire': 'Mamour walla sonko Amna kou thi am probléme Kane ? Xam ! Na saff na saffa saff rék Et nous on va compter Les coups'}


In [47]:
df["commentairesArticle"] = df["commentairesArticle"].apply(lambda x : [transformer_commentaire(comment) for comment in x] )

In [48]:
df["commentairesArticle"] = df["commentairesArticle"].apply(lambda x :  sort_comment_list(x))

#### Nos données contiennent elles des valeurs en double ?

In [100]:
# taille avant suppresion des doublons 
df.shape # 9063

(9063, 21)

In [98]:
### Critère de detection des doublons
sub = ['sourceArticle','titreArticle', 'resumeArticle', 'auteurArticle','contenuArticle', 'datePublicationArticle']

In [101]:
df = df.drop_duplicates(subset=sub, keep='first')

In [102]:
# taille avant suppresion des doublons 
df.shape  # 7304 soit 1759 doublons

(7304, 21)

In [103]:
# Utilisez la méthode to_json() pour sauvegarder le DataFrame au format JSON
df.to_json("data/processed_baseArticles2.json", orient="records") #, lines=True

In [104]:
with open("data/processed_baseArticles2.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Parcourir les objets JSON et supprimer les échappements des "/"
for item in data:
    for key, value in item.items():
        if isinstance(value, str):
            item[key] = value.replace("\\/", "/")

# Réenregistrer le fichier JSON sans les échappements des "/"
with open("data/processed_baseArticles_final.json", "w", encoding="utf-8") as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

#### Stockage des données dans MongoDB

In [1]:
import pymongo

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["articlesdb"] # Création de la db
collection = db["articles"] # Création de la collection

In [3]:
import json

with open("data/processed_baseArticles_final.json", "r", encoding="utf-8") as file:
    data = json.load(file)

collection.insert_many(data) # Chargement des données

In [5]:
client.close()